In [ ]:
## 以下脚本用于部署 Allin1 WebUI GUI

## 代码及Web环境部署

In [8]:
%%bash
sudo chown -R ec2-user:ec2-user .
docker logout public.ecr.aws

Removing login credentials for public.ecr.aws


In [ ]:
%%bash
git clone --recurse-submodules -j8 https://YOUR_GIT_PRIVATE_KEY@github.com/xieyongliang/all-in-one-ai.git
git pull --recurse-submodules -j8

In [ ]:
#准备相关代码,此步骤执行时间较长,请耐心等待.

In [9]:
import boto3
import os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
os.chdir('/home/ec2-user/SageMaker/Allin1-Deploy/all-in-one-ai/deployment')
s3uri = os.getenv('s3uri')
cmd = "/bin/sh ./build_and_deploy.sh s3://%s %s stable-diffusion-webui >>bb.log" % (
    s3uri, region_name)
print(cmd)
os.system(cmd)

set -e
#

# This script should be run from the repo's backend directory
#
#
# Get s3uri and region from command line
s3uri=$1
region=$2

# Get reference for all important folders
backend_dir="$PWD"
project_dir="$backend_dir/.."
build_dist_dir="$backend_dir/build/codes"
source_dir="$backend_dir/src"
build_dir="$backend_dir/build/tmp"

echo "------------------------------------------------------------------------------"
echo "[Init] Clean old dist folders"
echo "------------------------------------------------------------------------------"
echo "rm -rf $build_dist_dir"
rm -rf $build_dist_dir
echo "mkdir -p $build_dist_dir"
mkdir -p $build_dist_dir

mkdir -p ${build_dir}/

echo "------------------------------------------------------------------------------"
echo "[Rebuild] all_in_one_ai_helper lambda functions"
echo "------------------------------------------------------------------------------"

echo ${source_dir}
cd ${source_dir}/all_in_one_ai_helper
rm -r ${build_dir}

mkdir -p ${buil

In [137]:
%%bash
all_az=()
cfn=$(aws s3 presign s3://$s3uri/templates/all-in-one-ai-webui-main.yaml | cut -d '?' -f 1)
az_per_region=$(aws ec2 describe-availability-zones --query 'AvailabilityZones[*].[ZoneName]' --output text | sort)                                                                             
  while read -r az; do                                                             
     all_az+=($az)                                                                  
  done <<< "$az_per_region" 

aws cloudformation create-stack --stack-name allin1ai-gui \
    --template-url $cfn \
    --parameters \
    ParameterKey=WebUIDomainName,ParameterValue="" \
    ParameterKey=UserPoolDomain,ParameterValue="" \
    ParameterKey=DomainName,ParameterValue="" \
    ParameterKey=CognitoRegion,ParameterValue="" \
    ParameterKey=UserPool,ParameterValue="" \
    ParameterKey=UserPoolClient,ParameterValue="" \
    ParameterKey=Certificate,ParameterValue="" \
    ParameterKey=AvailabilityZones,ParameterValue=${all_az[0]}\\,${all_az[1]}\\,${all_az[2]} \
    ParameterKey=S3Bucket,ParameterValue=$s3uri \
    ParameterKey=S3Key,ParameterValue="" \
    --capabilities CAPABILITY_NAMED_IAM

{
    "StackId": "arn:aws:cloudformation:ap-northeast-1:544592066775:stack/allin1ai-gui1/d37c0430-02cf-11ee-a035-0eb3fa885329"
}


In [22]:
!aws cloudformation describe-stacks --stack-name allin1ai-gui --output json --query 'Stacks[0].StackStatus'

"CREATE_IN_PROGRESS"


In [20]:
WebUI_URL="$(aws cloudformation describe-stacks --stack-name allin1ai-api --output text --query 'Stacks[0].Outputs[?OutputKey==`WebUIPortalUrl`].OutputValue')"

In [152]:
!echo $WebUI_URL

https://sl5h9soz0e.execute-api.eu-central-1.amazonaws.com/Prod


## 上传模型

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

如果启动时只选机型和数量，则可以通过动态加载默认S3中的模型⽂件。
Sagemaker节点启动成功后, 每30s从默认的s3 位置s3://sagemaker-{region_name}-
{account_id}/stable-diffusion-webui/models/同步模型⽂件到sagemaker节点的本地⽬录下。⽀
持Stable-diffusion, Lora, ControlNet模型的动态加载同步。如：

• s3://$bucket/stable-diffusion-webui/models/Stable-diffusion/

• s3://$bucket/stable-diffusion-webui/models/ControlNet/

• s3://$bucket/stable-diffusion-webui/models/Lora/


In [140]:
!echo $bucket

sagemaker-ap-northeast-1-544592066775


In [ ]:
#如果没有上述文件夹可以通过如下命令创建

In [ ]:
%bash
aws s3api put-object --bucket $bucket --key stable-diffusion-webui/models/Stable-diffusion/
aws s3api put-object --bucket $bucket --key stable-diffusion-webui/models/ControlNet/
aws s3api put-object --bucket $bucket --key stable-diffusion-webui/models/Lora/

## 登录Web UI

In [ ]:
%%bash
Admin_info="$(aws cloudformation describe-stacks --stack-name allin1ai-GUI --output text --query 'Stacks[0].Outputs[?OutputKey==`Administratorlogin`].OutputValue'| cut -d ":" -f 7)"

In [ ]:
#获取登录密码

In [ ]:
%%bash
cd /home/ec2-user/SageMaker/Allin1-Deploy/all-in-one-ai/sagemaker/stable-diffusion-webui/tools
./secretmanager.sh $Admin_info $region_name get

In [ ]:
#登录

In [ ]:
!echo $WebUI_URL